In [1]:
!pip install rnnmorph

In [2]:
import pandas as pd
import numpy as np

from rnnmorph.predictor import RNNMorphPredictor

import re

def is_phrase(string):
    return " " in string

def is_word(string):
    pattern = re.compile("[ЁёА-я]+")
    return bool(pattern.fullmatch(string))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
DATA_FILE = '/content/drive/My Drive/CSC/практика_весна22/data/prepared_to_tag.csv'
SAVE_TO = '/content/drive/My Drive/CSC/практика_весна22/data/rnnmorph.csv'
tagged_cols = ["rnn_stim_tag", "rnn_word_tag"]

## Глава 3. NN

`RNNMorph`

In [5]:
data = pd.read_csv(DATA_FILE, keep_default_na = False, na_values = [''])
data.head(15)

,id,stim,word,phrase,auto_tag
0,1,абсолютный,-,абсолютный -,NULL
1,1,абсолютный,0,абсолютный 0,NULL
2,1,абсолютный,100%,абсолютный 100%,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,NULL
4,1,абсолютный,max,абсолютный max,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,NaN
6,1,абсолютный,авторитет,абсолютный авторитет,NaN
7,1,абсолютный,агент,абсолютный агент,NaN
8,1,абсолютный,адреналин,абсолютный адреналин,NaN
9,1,абсолютный,анализ,абсолютный анализ,NaN


Создадим колонки для определения частей речи rnnMorph'ым

In [6]:
for c in tagged_cols:
    data[c] = None
data[tagged_cols[1]] = data.auto_tag
data = data.drop(['auto_tag'], axis = 1)
data.head(15)

,id,stim,word,phrase,rnn_stim_tag,rnn_word_tag
0,1,абсолютный,-,абсолютный -,None,NULL
1,1,абсолютный,0,абсолютный 0,None,NULL
2,1,абсолютный,100%,абсолютный 100%,None,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,None,NULL
4,1,абсолютный,max,абсолютный max,None,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,None,NaN
6,1,абсолютный,авторитет,абсолютный авторитет,None,NaN
7,1,абсолютный,агент,абсолютный агент,None,NaN
8,1,абсолютный,адреналин,абсолютный адреналин,None,NaN
9,1,абсолютный,анализ,абсолютный анализ,None,NaN


### DeepPavlov works

In [7]:
predictor = RNNMorphPredictor(language="ru")

In [8]:
def rnnmorph_tag(phrase):
  return ' '.join([pred.pos for pred in predictor.predict(phrase.split(' '))])

In [9]:
%%time
fun = rnnmorph_tag
dp_tags = data[data.word.apply(is_word)].phrase.apply(fun)
data.loc[data.word.apply(is_word), tagged_cols[0]] = dp_tags.apply(lambda x: x.split(' ')[0])
data.loc[data.word.apply(is_word), tagged_cols[1]] = dp_tags.apply(lambda x: x.split(' ')[1])
data.loc[~data.word.apply(is_word), tagged_cols[0]] = data.loc[~data.word.apply(is_word), "stim"].apply(fun)
data[:20]

CPU times: user 2h 32min 48s, sys: 3min, total: 2h 35min 48s
Wall time: 2h 31min 28s


In [10]:
data.to_csv(SAVE_TO, index=False)